# Creacion del modelo 

In [1]:
# Importo las librerias correspondientes para realizar el analisis

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import hstack, csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


### 1. Obtencion y organizacion de los datos

In [2]:
# Obtengo los datos del dataset creado en el EDA
dataframe_unido_modelo = pd.read_parquet('Datasets/dataset_unido_EDA')

# Reemplazo los valores nulos en las columnas que voy a vectorizar
dataframe_unido_modelo['name_genre'] = dataframe_unido_modelo['name_genre'].fillna('')
dataframe_unido_modelo['cast_name_actor'] = dataframe_unido_modelo['cast_name_actor'].fillna('')
dataframe_unido_modelo['crew_name_member'] = dataframe_unido_modelo['crew_name_member'].fillna('')
dataframe_unido_modelo['overview'] = dataframe_unido_modelo['overview'].fillna('')

### 2. Vectorización de las características numéricas

In [3]:
# Normalizo las características numéricas que voy a usar
caracteristicas_numericas = ['budget', 'revenue', 'vote_count', 'popularity',]
escala = StandardScaler()
caracteristicas_numericas_normalizadas = escala.fit_transform(dataframe_unido_modelo[caracteristicas_numericas])


### 3. Vectorización de las características categóricas

In [4]:
# Realizo la vectorizacion del género de la película
vectorizar_genero = TfidfVectorizer(tokenizer=lambda x: x.split(','))
matrix_tfidf_genero = vectorizar_genero.fit_transform(dataframe_unido_modelo['name_genre'])

# Realizo la vectorizacion de los actores 
vectorizar_actor = TfidfVectorizer(tokenizer=lambda x: x.split(','))
matrix_tfidf_actor = vectorizar_actor.fit_transform(dataframe_unido_modelo['cast_name_actor'])

# Filtro solo los directores y vectorizo
dataframe_unido_modelo['director'] = dataframe_unido_modelo.apply(lambda x: x['crew_name_member'] if x['crew_job'] == 'Director' else '', axis=1)
vectorizar_director = TfidfVectorizer(tokenizer=lambda x: x.split(','))
matrix_tfidf_director = vectorizar_director.fit_transform(dataframe_unido_modelo['director'])

# Realizo la vectorizacion del overview
vectorizar_overview = TfidfVectorizer(stop_words='english')
matrix_tfidf_overview = vectorizar_overview.fit_transform(dataframe_unido_modelo['overview'])


d:\DOCUMENTOS\DATA_SCIENCE\Proyecto Machine Learning\entorno_virtual\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
d:\DOCUMENTOS\DATA_SCIENCE\Proyecto Machine Learning\entorno_virtual\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 4. Combinación de características:


In [5]:
# primero convierto la matriz numerica escalada a una matriz dispersa
matrix_numerica_escalada = csr_matrix(caracteristicas_numericas_normalizadas)

# Pondero las matrices para agregar mas peso a algunas caracteristicas
peso_del_genero = 6.0
peso_del_actor = 3.0
peso_del_director = 2.0
peso_del_overview = 3.0

peso_del_genero_matrix = peso_del_genero * matrix_tfidf_genero
peso_del_actor_matrix = peso_del_actor * matrix_tfidf_actor
peso_del_director_matrix = peso_del_director * matrix_tfidf_director
peso_del_overview_matrix = peso_del_overview * matrix_tfidf_overview

# Combino todas las matrices ponderadas
caracteristicas_combinadas = hstack([
    matrix_numerica_escalada,
    peso_del_genero_matrix,
    peso_del_actor_matrix,
    peso_del_director_matrix,
    peso_del_overview_matrix
])

### 5. Realizar el calculo de la similitud del coseno

In [6]:
# Me aseguro de que 'features_combined' es una matriz dispersa (csr_matrix)
if not isinstance(caracteristicas_combinadas, csr_matrix):
    caracteristicas_combinadas = csr_matrix(caracteristicas_combinadas)

# Reducir la dimensionalidad
svd = TruncatedSVD(n_components=100)  # Ajusta el número de componentes según sea necesario
caracteristicas_reducidas = svd.fit_transform(caracteristicas_combinadas)

# Calcular similitud del coseno con la versión reducida
similitud_del_coseno = cosine_similarity(caracteristicas_reducidas)

In [7]:
def recommend_movies(title, similitud_del_coseno=similitud_del_coseno):
    """
    Recomienda peliculas similares a una pelicula dada basada en la similitud del coseno.

    Args:
        title: El título de la película para la cual se desean obtener recomendaciones.
        similitud_del_coseno: La matriz de similitud del coseno entre películas.

    Returns:
       
        Una serie de 5 títulos de películas recomendadas que son más similares a la película dada,
        o un mensaje indicando que el título no se encuentra disponible.
    """
    # Normalizar el título para comparar sin importar mayúsculas/minúsculas
    title = title.lower()
    
    # Convertir todos los títulos en minúsculas para la comparación
    dataframe_unido_modelo['title_lower'] = dataframe_unido_modelo['title'].str.lower()
    
    # Verificar la existencia del título en el DataFrame
    if title not in dataframe_unido_modelo['title_lower'].values:
        return f"El título '{title}' no se encuentra disponible en la base de datos."
    
    # Obtener el índice de la película dada
    idx = dataframe_unido_modelo[dataframe_unido_modelo['title_lower'] == title].index[0]
    
    # Manejar matrices dispersas y densas
    if isinstance(similitud_del_coseno, csr_matrix):
        # Convertir a denso si es necesario
        sim_scores = similitud_del_coseno[idx].toarray().flatten()
    else:
        # Si ya es denso, simplemente usa el índice
        sim_scores = similitud_del_coseno[idx]
    
    # Enumerar y ordenar las similitudes
    sim_scores = list(enumerate(sim_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Tomar las 5 películas más similares
    sim_scores = sim_scores[1:6]  # Excluye la película misma
    
    # Obtener los índices de las películas recomendadas
    movie_indices = [i[0] for i in sim_scores]
    
    # Retornar los títulos de las películas recomendadas
    return dataframe_unido_modelo[['title']].iloc[movie_indices]





In [8]:
# Verificacion en diferentes peliculas 

print("Recomendaciones para 'The Avengers':")
print(recommend_movies('The Avengers'))

print("\nRecomendaciones para 'Avatar':")
print(recommend_movies('Avatar'))

print("\nRecomendaciones para 'Men in Black':")
print(recommend_movies('Men in Black'))

print("\nRecomendaciones para 'toy story':")
print(recommend_movies('toy story'))

print("\nRecomendaciones para 'john wick: chapter 2':")
print(recommend_movies('john wick: chapter 2'))

print("\nRecomendaciones para 'titanic':")
print(recommend_movies('titanic'))

print("\nRecomendaciones para 'saw':")
print(recommend_movies('saw'))

print("\nLa pelicula buscada no se encuentra en la base de datos:")
print(recommend_movies('bee'))



Recomendaciones para 'The Avengers':
               title
6024     The Missing
9807     The Reaping
4136    Original Sin
2893  Reindeer Games
1261       The Saint

Recomendaciones para 'Avatar':
                              title
14220                  The Avengers
20812    Captain America: Civil War
19650                Jurassic World
32238               Despicable Me 3
30528  Rogue One: A Star Wars Story

Recomendaciones para 'Men in Black':
                                     title
7223                       The Incredibles
3129                             Gladiator
11958                      Sherlock Holmes
18074  Captain America: The Winter Soldier
15268                              Skyfall

Recomendaciones para 'toy story':
                title
12595   Despicable Me
4578          Ice Age
2705      Toy Story 2
4300   Monsters, Inc.
541           Aladdin

Recomendaciones para 'john wick: chapter 2':
                                title
267                      Pulp Fiction
1405